In [1]:
import os
import sys
sys.path.append('../ideepcolor')
from data import colorize_image as CI
import caffe
import skimage.io
import skimage.color
from skimage import img_as_float
import matplotlib.pyplot as plt
import numpy as np
import segment
import prepare_data

%matplotlib inline

In [2]:
orig_img_prefix = '../img/original/'
bw_img_prefix = '../img/bw/'
auto_img_prefix = '../img/auto/'
res_img_prefix = '../img/result/'
user_img_prefix = '../img/user/'
seg_img_prefix = '../img/segment/'

img_urls = prepare_data.generate_bw_images_and_list()

In [3]:
def loadAutoModel():
    return 'python ./colorize.py -img_in %s -img_out %s --gpu -1 --caffemodel ./models/colorization_release_v1.caffemodel'

In [4]:
def loadUserModel():
    os.chdir('../ideepcolor')
    gpu_id = -1 # cpu
    colorModel = CI.ColorizeImageCaffe(Xd=256)
    colorModel.prep_net(gpu_id,'./models/reference_model/deploy_nodist.prototxt','./models/reference_model/model.caffemodel')
    os.chdir('../src')
    return colorModel

In [5]:
# Does not return anything, just writes the colored image
def auto_colorize(img_url, model):
    os.chdir('../colorization')
    os.system(model % (bw_img_prefix + img_url, auto_img_prefix + img_url))
    temp_img = skimage.io.imread(auto_img_prefix + img_url)
    skimage.io.imsave(auto_img_prefix + img_url, temp_img[:,:,:3]) # only want RGB channels, not A
    os.chdir('../src')

In [6]:
# returns a list of ((r,c), (L,a,b)) tuples for (row, col) points to color in LAB space
# r and c are in [0, 1)
def get_color_points(img_url):
    return segment.get_color_points(img_url)

In [7]:
# Does not return anything, writes the final result
def user_driven_color(img_url, model, points):
    model.load_image(bw_img_prefix + img_url) 
    mask = np.zeros((1,256,256))
    input_ab = np.zeros((2,256,256))
    
    # user-driven without input points
    img_out = model.net_forward(input_ab, mask)
    img_out_fullres = model.get_img_fullres()
    skimage.io.imsave(user_img_prefix + img_url, img_out_fullres)
    
    for ((r, c), (L, a, b)) in points:
        r = int(r*256)
        c = int(c*256)
        mask[0, r, c] = 1
        input_ab[0, r, c] = a
        input_ab[1, r, c] = b
    
    # user-driven with input points (final result)
    img_out = model.net_forward(input_ab, mask)
    img_out_fullres = model.get_img_fullres()
    skimage.io.imsave(res_img_prefix + img_url, img_out_fullres)

In [ ]:
autoModel = loadAutoModel()
userModel = loadUserModel()

ColorizeImageCaffe instantiated
gpu_id = -1, net_path = ./models/reference_model/deploy_nodist.prototxt, model_path = ./models/reference_model/model.caffemodel


In [ ]:
for img_url in img_urls:
    auto_colorize(img_url, autoModel)
    points = get_color_points(img_url)
    user_driven_color(img_url, userModel, points)

/usr/local/lib/python2.7/dist-packages/skimage/color/colorconv.py:985: UserWarning: Color data out of range: Z < 0 in 2 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
/usr/local/lib/python2.7/dist-packages/skimage/color/colorconv.py:985: UserWarning: Color data out of range: Z < 0 in 1 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
/usr/local/lib/python2.7/dist-packages/skimage/color/colorconv.py:985: UserWarning: Color data out of range: Z < 0 in 19 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
/usr/local/lib/python2.7/dist-packages/skimage/color/colorconv.py:985: UserWarning: Color data out of range: Z < 0 in 7 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)


In [ ]:
def show_images(img_url):
    auto_image = skimage.io.imread(auto_img_prefix+img_url)
    user_image = skimage.io.imread(user_img_prefix+img_url)
    res_image = skimage.io.imread(res_img_prefix+img_url)
    #bw_image = skimage.color.gray2rgb(skimage.io.imread(bw_img_prefix+img_url))
    seg_image = skimage.io.imread(seg_img_prefix+img_url[:-3]+'png')
    orig_image = skimage.io.imread(orig_img_prefix+img_url)
    plt.imshow(np.concatenate((auto_image, user_image, res_image, seg_image, orig_image), axis=1))

In [ ]:
plt.figure(figsize=(18, 3*len(img_urls)))
for index, img_url in enumerate(img_urls):
    plt.subplot(len(img_urls), 1, index+1)
    show_images(img_url)
    plt.axis('off')